In [3]:
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit.algorithms.gradients import ParamShiftEstimatorGradient

ImportError: cannot import name '_param_shift_preprocessing' from 'qiskit.algorithms.gradients.utils' (/Users/jul/Qiskit/qiskit-terra/qiskit/algorithms/gradients/utils.py)

In [9]:
from collections import defaultdict
from qiskit import transpile
from qiskit.circuit import ParameterExpression, Parameter

In [10]:
p = ParameterVector("x", 3)
circuit = QuantumCircuit(1)
circuit.rx(p[0], 0)
circuit.ry(p[1] * p[1], 0)
circuit.rx(p[0] + 2 * p[1], 0)
supported_gates = ["rx", "ry"]

In [11]:
circuit2 = transpile(circuit, basis_gates=supported_gates, optimization_level=0)
gradient_circuit = circuit2.copy_empty_like(f"{circuit.name}_gradient")
parameter_map = defaultdict(list)
gradient_parameter_map = {}
num_gradient_parameters = 0
for instruction, qargs, cargs in circuit2.data:
    if instruction.is_parameterized():
        new_inst_params = []
        for angle in instruction.params:
            new_parameter = Parameter(f"gθ{num_gradient_parameters}")
            new_inst_params.append(new_parameter)
            num_gradient_parameters += 1
            for parameter in angle.parameters:
                parameter_map[parameter].append((new_parameter, angle.gradient(parameter)))
            gradient_parameter_map[new_parameter] = angle
        instruction.params = new_inst_params
    gradient_circuit.append(instruction, qargs, cargs)
# For the global phase
gradient_circuit.global_phase = circuit2.global_phase
if isinstance(gradient_circuit.global_phase, ParameterExpression):
    substitution_map = {}
    for parameter in gradient_circuit.global_phase.parameters:
        if parameter in parameter_map:
            substitution_map[parameter] = parameter_map[parameter][0][0]
        else:
            new_parameter = Parameter(f"gθ{num_gradient_parameters}")
            substitution_map[parameter] = new_parameter
            parameter_map[parameter].append(new_parameter, 1)
            num_gradient_parameters += 1
    gradient_circuit.global_phase = gradient_circuit.global_phase.subs(substitution_map)

In [13]:
gradient_circuit.draw()

┌─────────┐┌─────────┐┌─────────┐
q: ┤ Rx(gθ0) ├┤ Ry(gθ1) ├┤ Rx(gθ2) ├
   └─────────┘└─────────┘└─────────┘

In [14]:
parameter_map

defaultdict(list,
            {ParameterVectorElement(x[0]): [(Parameter(gθ0), 1.0),
              (Parameter(gθ2), 1.0)],
             ParameterVectorElement(x[1]): [(Parameter(gθ1),
               ParameterExpression(2*x[1])),
              (Parameter(gθ2), 2.0)]})

In [15]:
gradient_parameter_map

{Parameter(gθ0): ParameterVectorElement(x[0]),
 Parameter(gθ1): ParameterExpression(x[1]**2),
 Parameter(gθ2): ParameterExpression(x[0] + 2*x[1])}